In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/TIU-con/LED-Filtering-JAENS

/content/drive/MyDrive/TIU-con/LED-Filtering-JAENS


In [3]:
!pip install datasets
!pip install tqdm
!pip install bert-score
from datasets import load_dataset, load_metric
from tqdm import trange
from tqdm import tqdm
import time
from bert_score import score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00


In [4]:
#Checking predicted results csv files total rows.
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/TIU-con/LED-Filtering-JAENS/LED_Jaens_Filtering_result.csv")
#data

In [5]:
data_final=data[['article', 'abstract', 'predicted_abstract']]
data_final['abstract'][0]

" research on the implications of anxiety in parkinson 's disease ( pd ) has been neglected despite its prevalence in nearly 50% of patients and its negative impact on quality of life . \n previous reports have noted that neuropsychiatric symptoms impair cognitive performance in pd patients ; however , to date , no study has directly compared pd patients with and without anxiety to examine the impact of anxiety on cognitive impairments in pd . \n this study compared cognitive performance across 50 pd participants with and without anxiety ( 17 pda+ ; 33 pda ) , who underwent neurological and neuropsychological assessment . \n group performance was compared across the following cognitive domains : simple attention / visuomotor processing speed , executive function ( e.g. , set - shifting ) , working memory , language , and memory / new verbal learning . \n results showed that pda+ performed significantly worse on the digit span forward and backward test and part b of the trail making tas

In [6]:
import csv
#finding BERT score
l=len(data_final)
with open('test_bertscore.csv', mode='w') as csv_file:
  fieldnames = ['abstract','BERT-P','BERT-R','BERT-F1']
  #fieldnames = ['Article', 'Actual Summary','Predective Summary']
  writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
  writer.writeheader()

  #variables to calculate sum of BERT score
  sum_BERT_P=0
  sum_BERT_R=0
  sum_BERT_F1=0
  for i in tqdm(range(l)):

    sample_row = data_final.iloc[i]
    #abstract=sample_row['abstract']   Avoid to write article becoz contain more text
    model_output = sample_row['predicted_abstract']
    ref_output = sample_row['abstract']


    #calculating bert-score
    BERT_P,BERT_R,BERT_F1= score([model_output], [ref_output], lang="en")
    #BERT_P,BERT_R,BERT_F1= score([model_output], [ref_output], lang="en",verbose=True)

    BERT_P=round(BERT_P.item(),4)
    BERT_R=round(BERT_R.item(),4)
    BERT_F1=round(BERT_F1.item(),4)

    writer.writerow({'abstract':ref_output, 'BERT-P':BERT_P,'BERT-R':BERT_R,'BERT-F1':BERT_F1})

    sum_BERT_P=sum_BERT_P+BERT_P
    sum_BERT_R=sum_BERT_R+BERT_R
    sum_BERT_F1=sum_BERT_F1+BERT_F1

 #finding average Bert score
  Avg_BERT_P =sum_BERT_P/l
  Avg_BERT_R =sum_BERT_R/l
  Avg_BERT_F1 =sum_BERT_F1/l

  #print average  bert score
  print('\n')
  Avg_BERT_P = round(Avg_BERT_P,4)
  Avg_BERT_R = round(Avg_BERT_R,4)
  Avg_BERT_F1 = round(Avg_BERT_F1,4)
  print("Avg_BERT_P",Avg_BERT_P)
  print("Avg_BERT_R",Avg_BERT_R)
  print("Avg_BERT_F1", Avg_BERT_F1)
#next one
# write all results in a text file
with open("BERTScore.txt","w") as f:
  f.write("Avg_BERT_P:" + str(Avg_BERT_P) +'\n')
  f.write("Avg_BERT_R:" + str(Avg_BERT_R) + "\n")
  f.write("Avg_BERT_F1:" + str(Avg_BERT_F1) + "\n")


  0%|          | 0/240 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 1/240 [00:59<3:58:23, 59.85s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 2/240 [01:10<2:03:11, 31.06s/it]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|▏         | 3/240 [01



Avg_BERT_P 0.8217
Avg_BERT_R 0.846
Avg_BERT_F1 0.8335


In [ ]:
#automatically delete runtime
from google.colab import runtime
runtime.unassign()